In [24]:
import pandas as pd

data = pd.read_csv('./data/lab1/data.csv', sep=';')['Survival'].to_frame()
survival_cout = data[data>1].count()
print(data[data >1].count())
print(data[data == 1].count())
print(data.count())
print('test1')

Survival    86
dtype: int64
Survival    140
dtype: int64
Survival    226
dtype: int64
